# Shakespeare Model Testing

Goals:
- Load Data from data folder
- Process Data for model
- Run Training for some Generations

## Load Data

In [1]:
from pathlib import Path
import os
import pickle
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
base_directory = Path(os.getcwd()).parent.parent
shakespeare_data_root_directory = base_directory / "data" / "shakespeare"

sample = 0
shakespeare_data_sample_directory = shakespeare_data_root_directory / f"{sample}"

shakespeare_sample_test_file = shakespeare_data_sample_directory / "test.pickle"
shakespeare_sample_train_file = shakespeare_data_sample_directory / "train.pickle"
shakespeare_sample_validation_file = shakespeare_data_sample_directory / "val.pickle"

In [3]:
shakespeare_sample_train_data = pd.read_pickle(shakespeare_sample_train_file)
shakespeare_sample_test_data = pd.read_pickle(shakespeare_sample_test_file)
shakespeare_sample_validation_data = pd.read_pickle(shakespeare_sample_validation_file)

## Create Model

In [25]:
from sources.models.shakespeare.shakespeare_client_model import ShakespeareClientModel

client_model = ShakespeareClientModel(0.1)
model = client_model.get_model()

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 80)]              0         
                                                                 
 embedding_8 (Embedding)     (None, 80, 8)             640       
                                                                 
 rnn_2 (RNN)                 (None, 256)               796672    
                                                                 
 dense_2 (Dense)             (None, 80)                20560     
                                                                 
 softmax_2 (Softmax)         (None, 80)                0         
                                                                 
Total params: 817,872
Trainable params: 817,872
Non-trainable params: 0
_________________________________________________________________


## Process Data, Compile Model and Fit

In [31]:
client_model.process_x(shakespeare_sample_train_data["x"]).shape

(3514, 80)

In [32]:
model.compile(optimizer=client_model.get_default_optimizer(),
             loss=client_model.get_default_loss_function(),
             metrics=['accuracy', "mse"])

len(shakespeare_sample_train_data["x"])

3514

In [33]:
model.fit(client_model.process_x(shakespeare_sample_train_data["x"]),
          client_model.process_y(shakespeare_sample_train_data["y"]),
          validation_data= (client_model.process_x(shakespeare_sample_test_data["x"]),
                            client_model.process_y(shakespeare_sample_test_data["y"])),
          epochs=1)

110/110 [==============================] - 33s 285ms/step - loss: 4.2717 - accuracy: 0.1881 - mse: 0.0123 - val_loss: 4.1367 - val_accuracy: 0.1793 - val_mse: 0.0122

In [34]:
model.evaluate(
    x=client_model.process_x(shakespeare_sample_validation_data["x"]),
    y=client_model.process_y(shakespeare_sample_validation_data["y"]),
)

37/37 [==============================] - 2s 50ms/step - loss: 4.1471 - accuracy: 0.1783 - mse: 0.0122

[4.1470842361450195, 0.17832764983177185, 0.012249120511114597]